In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
with open("/home/lwi/data/MLProject/adjacency_matrices.pkl", "rb") as file:
    adj = pickle.load(file)

In [3]:
xTrain = pd.read_csv("~/data/MLProject/xTrain.csv", index_col=0)
xTest = pd.read_csv("~/data/MLProject/xTest.csv", index_col=0)

In [4]:
xTrain['sample_id'] = 21 * xTrain['cluster'] + xTrain['day']
xTrain.set_index('sample_id', inplace=True)
xTrain.sort_index(inplace=True)
xTest['sample_id'] = 21 * xTest['cluster'] + xTest['day']
xTest.set_index('sample_id', inplace=True)
xTest.sort_index(inplace=True)

In [5]:
xTrain.drop(columns=['id', 'day', 'asset'], inplace=True)
xTest.drop(columns=['id', 'day', 'asset'], inplace=True)

In [6]:
xTrain['size_cluster'] = xTrain['cluster'].transform(lambda nb: adj[nb].shape[0])
xTrain['prop_correlated'] = xTrain['cluster'].transform(lambda nb: adj[nb].sum().sum() / adj[nb].size)
xTest['size_cluster'] = xTest['cluster'].transform(lambda nb: adj[nb].shape[0])
xTest['prop_correlated'] = xTest['cluster'].transform(lambda nb: adj[nb].sum().sum() / adj[nb].size)

In [9]:
xTrainMean = xTrain.drop(columns='cluster').groupby('sample_id').mean()
xTrainStd = xTrain.drop(columns=['cluster', 'size_cluster', 'prop_correlated']).groupby('sample_id').std()
xTrainStd = xTrainStd.rename(columns={att: att+"_std" for att in xTrainStd.columns})
xTrainMedian = xTrain.drop(columns=['cluster', 'size_cluster', 'prop_correlated']).groupby('sample_id').median()
xTrainMedian = xTrainMedian.rename(columns={att: att+"_med" for att in xTrainMedian.columns})
xTrainMean.join(xTrainStd).join(xTrainMedian).fillna(0).to_csv("~/data/MLProject/xTrainFinal.csv")

In [10]:
xTestMean = xTest.drop(columns='cluster').groupby('sample_id').mean()
xTestStd = xTest.drop(columns=['cluster', 'size_cluster', 'prop_correlated']).groupby('sample_id').std()
xTestStd = xTestStd.rename(columns={att: att+"_std" for att in xTestStd.columns})
xTestMedian = xTest.drop(columns=['cluster', 'size_cluster', 'prop_correlated']).groupby('sample_id').median()
xTestMedian = xTestMedian.rename(columns={att: att+"_med" for att in xTestMedian.columns})
xTestMean.join(xTestStd).join(xTestMedian).fillna(0).to_csv("~/data/MLProject/xTestFinal.csv")